In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import pandas as pd
from tqdm import tqdm
from keras.preprocessing import image


In [ ]:
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
print(os.getcwd())
save_dir = os.path.join(os.getcwd(), 'saved_models')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks


In [ ]:
train_labels = pd.read_csv('/content/gdrive/MyDrive/Xray dataset/labels_train.csv')
test_labels = pd.read_csv('/content/gdrive/MyDrive/Xray dataset/sample_submission.csv')
dataset_url = '/content/gdrive/MyDrive/Xray dataset'
directory_train = '/content/gdrive/MyDrive/Xray dataset/train_images/'
directory_test = '/content/gdrive/MyDrive/Xray dataset/test_images/'
trainingNPY = '/content/gdrive/MyDrive/pneumonia/training.npy'
testingNPY = '/content/gdrive/MyDrive/pneumonia/testing.npy'
training224= '/content/gdrive/MyDrive/Colab Notebooks/training224.npy'
testing224= '/content/gdrive/MyDrive/Colab Notebooks/testing224.npy'

In [ ]:
#x_train=np.load(trainingNPY)
#x_test=np.load(testingNPY)

In [ ]:
training_set_image = [ ]

for i in tqdm(range(train_labels.shape[0])):
  # img = image.load_img(directory_train+train_labels['file_name'][i], target_size=(128,128,3))
  img = image.load_img(directory_train+train_labels['file_name'][i], target_size=(224,224,3))
  img = image.img_to_array(img)
  img = img/255.0
  training_set_image.append(img)
x_train = np.array(training_set_image,dtype="float32")
print(x_train)

100%|██████████| 4672/4672 [33:30<00:00,  2.32it/s]


[[[[0.19215687 0.19215687 0.19215687]
   [0.19215687 0.19215687 0.19215687]
   [0.19215687 0.19215687 0.19215687]
   ...
   [0.10980392 0.10980392 0.10980392]
   [0.10196079 0.10196079 0.10196079]
   [0.09411765 0.09411765 0.09411765]]

  [[0.2        0.2        0.2       ]
   [0.20392157 0.20392157 0.20392157]
   [0.20392157 0.20392157 0.20392157]
   ...
   [0.11372549 0.11372549 0.11372549]
   [0.10196079 0.10196079 0.10196079]
   [0.09411765 0.09411765 0.09411765]]

  [[0.20784314 0.20784314 0.20784314]
   [0.20784314 0.20784314 0.20784314]
   [0.20784314 0.20784314 0.20784314]
   ...
   [0.13725491 0.13725491 0.13725491]
   [0.09019608 0.09019608 0.09019608]
   [0.10588235 0.10588235 0.10588235]]

  ...

  [[0.10980392 0.10980392 0.10980392]
   [0.11372549 0.11372549 0.11372549]
   [0.10588235 0.10588235 0.10588235]
   ...
   [0.16862746 0.16862746 0.16862746]
   [0.16862746 0.16862746 0.16862746]
   [0.16470589 0.16470589 0.16470589]]

  [[0.11372549 0.11372549 0.11372549]
   [0.1

In [ ]:
testing_set_image = [ ]

for i in tqdm(range(test_labels.shape[0])):
  img = image.load_img(directory_test+test_labels['file_name'][i], target_size=(224,224,3))
  img = image.img_to_array(img)
  img = img/255.0
  testing_set_image.append(img)
x_test = np.array(testing_set_image,dtype="float32")
print(x_test)

100%|██████████| 1168/1168 [07:43<00:00,  2.52it/s]


[[[[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.0627451  0.0627451  0.0627451 ]
   [0.03529412 0.03529412 0.03529412]
   [0.02745098 0.02745098 0.02745098]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.03921569 0.03921569 0.03921569]
   [0.03137255 0.03137255 0.03137255]
   [0.02745098 0.02745098 0.02745098]]

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.05098039 0.05098039 0.05098039]
   [0.03529412 0.03529412 0.03529412]
   [0.02745098 0.02745098 0.02745098]]

  ...

  [[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0. 

In [ ]:
print("Saving training image binary...")
np.save("training224",x_train) # Saves as "training.npy"
print("Done.")

Saving training image binary...
Done.


In [ ]:
print("Saving testing image binary...")
np.save("testing224",x_test) 
print("Done.")

Saving testing image binary...
Done.


In [ ]:
x_train=np.load(training224)
x_test=np.load(testing224)

In [ ]:
IMG_SIZE = 224
x_train = np.array(x_train).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
x_test = np.array(x_test).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [ ]:
y_train = np.array(train_labels.drop(['file_name'],axis=1))
y_train.shape
print(y_train)
y_test = np.array(test_labels.drop(['file_name'],axis=1))
y_test.shape
print(len(y_test))

[[1]
 [2]
 [2]
 ...
 [1]
 [1]
 [1]]
1168


In [ ]:
num_classes=3
depth=20

x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean



# Input image dimensions.
input_shape = x_train.shape[1:]

t_train = keras.utils.to_categorical(y_train, num_classes)
t_test = keras.utils.to_categorical(y_test, num_classes)



print('y_train (labels) shape:', y_train.shape)
print('t_train (one-hot rep) shape:', t_train.shape)

y_train (labels) shape: (4672, 1)
t_train (one-hot rep) shape: (4672, 3)


In [ ]:
tempx_train=x_train[500:]
tempt_train=t_train[500:] 
tempx_test=x_train[:500] 
tempt_test=t_train[:500]
tempy_train=y_train[500:]
tempy_test=y_test[:500]

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
datagen.fit(x_train)

In [ ]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=input_shape),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Flatten(),
    Dense(units=3, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=10,verbose=2)

Epoch 1/10
146/146 - 54s - loss: 27.0013 - accuracy: 0.3373
Epoch 2/10


KeyboardInterrupt: ignored

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model = Sequential()
for layer in vgg16_model.layers:
    print(layer)
    model.add(layer)

for layer in model.layers[:-1]:
    layer.trainable = False
model.pop()
model.add(Dense(units=3, activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(datagen.flow(tempx_train, tempt_train, batch_size=32),validation_data=(tempx_test,tempt_test),
          steps_per_epoch=len(tempx_train) / 32, epochs=10,verbose=1)

Epoch 1/10
130/130 [==============================] - 64s 484ms/step - loss: 1.0680 - accuracy: 0.4535 - val_loss: 1.0672 - val_accuracy: 0.4260
Epoch 2/10
130/130 [==============================] - 61s 464ms/step - loss: 0.9764 - accuracy: 0.5081 - val_loss: 1.0903 - val_accuracy: 0.4620
Epoch 3/10
130/130 [==============================] - 61s 464ms/step - loss: 0.9067 - accuracy: 0.5789 - val_loss: 1.1515 - val_accuracy: 0.4420
Epoch 4/10
130/130 [==============================] - 61s 469ms/step - loss: 0.8629 - accuracy: 0.6151 - val_loss: 1.1640 - val_accuracy: 0.4660
Epoch 5/10
130/130 [==============================] - 61s 466ms/step - loss: 0.8219 - accuracy: 0.6436 - val_loss: 1.2832 - val_accuracy: 0.4120
Epoch 6/10
130/130 [==============================] - 61s 464ms/step - loss: 0.7923 - accuracy: 0.6515 - val_loss: 1.2568 - val_accuracy: 0.4560
Epoch 7/10
130/130 [==============================] - 60s 462ms/step - loss: 0.7716 - accuracy: 0.6707 - val_loss: 1.3162 - val_ac